In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = 30

In [16]:
namedf = pd.read_csv('./class_names_2.txt', header = None)

In [17]:
namedf

,0
0,HIROKI
1,MISAKI
2,REINA
3,MOMOKO
4,MOEKA
5,RYOKO
6,STEVE
7,RENA
8,MAO
9,SHIORI


In [18]:
name_list = namedf.values.flatten()

In [2]:
num_days = 6
num_students = 23

students = 'abcdefghijklmnopqrstuvwxyz'[:num_students]

full_student_list = [
        char for char in students
]

speaker_roles = ['Prepared Speaker',
                'Impromptu Speaker',
                'Evaluator'
]

prepared_speakers = ['Prepared Speaker 1',
                     'Prepared Speaker 2',
                     'Prepared Speaker 3'
]

evaluators = [
    'Evaluator 1',
    'Evaluator 2',
    'Evaluator 3'
]

impromptu_speakers = [
    'Impromptu Speaker 1',
    'Impromptu Speaker 2',
    'Impromptu Speaker 3'    
]

speakers = ['Prepared Speaker 1',
           'Impromptu Speaker 1',
           'Evaluator 1',
            'Prepared Speaker 2',
            'Impromptu Speaker 2',
            'Evaluator 2',
            'Prepared Speaker 3',
            'Impromptu Speaker 3',
            'Evaluator 3'
]

leaders = [
            'Toastmaster',
            'President',
            'Table Topics Master',
            'General Evaluator'
]

auxiliary = ['Greeter',
            'Joke Master',
            'Timer',
            'Grammarian',
            'Word of the Day',
            'Ah Counter',
            'Ballot Counter',
            'Thought of the day',
            'Sergeant at arms',
            'Stand-in'
]

Constraints:
- Everyone has to do each of the three types of speeches once and only once
- Everyone should do one of the four leadership roles once.
    - repeats should preferentially be General Evaluator
- Everyone has one and only one job per day
- Those who do not have a speech or a leadership role fill in one of the auxiliary roles.

In [5]:
#everyone has to do each of the three types of speeches once and only once

# returns True if there are no repeat assignments for the given jobs.
# people can do multiple jobs, but not not the same job twice
def check_job_repeats(calendar, jobs = []):
    #check all columns if none given
    if len(jobs) == 0:
        jobs = calendar.columns
    result = True
    for job in jobs:
        if calendar[job].duplicated().sum() > 0:
            result = False
        #if any of the jobs have duplicates, finish the scan
        if not result:
            break
    return result


# nobody should have more than one job a day

# Checks if all the jobs on all the days of a dataframe are uniquely assigned --- no person has more than 1 job.
# returns True if so.
def check_day_repeats(calendar, days = []):
    if len(days) == 0:
        days = calendar.index
    result = True
    for row in days:
        if calendar.loc[row].duplicated().sum() > 0:
            result = False
        if not result:
            break
    return result

#returns true if there are no duplicates in the entire collection of job_list
def check_job_type_repeats(calendar, job_list, up_to_day):
    these_values = calendar.loc[:up_to_day, job_list].values.flatten()
    #print(these_values)
    try:
        u, c = np.unique(these_values, return_counts=True)
    except:
        print(f'Had a problem with these values:')
        print(these_values.reshape(-1,len(job_list)))
    dup = u[c > 1]
    return len(dup) == 0

#returns true if no prepared speaker /evaluator pairs occur more than once.
def check_speaker_evaluator_repeats(calendar):
    face_off_list = []
    speaker_evaluator_pairs = list(zip(prepared_speakers, evaluators))
    no_problems = True
    for day in calendar.index:
        for pair in speaker_evaluator_pairs:
            this_pair = set(calendar.loc[day, list(pair)])
            if this_pair in face_off_list:
                no_problems = False
            else:
                face_off_list.append(this_pair)
    return no_problems

Using [this stack overflow answer](https://stackoverflow.com/questions/11528078/determining-duplicate-values-in-an-array) to help with finding duplicates in the flattened numpy array.

Create a list of students to keep track of for each job. Each list 

Randomly assign students to the first set of jobs. For each job, remove that student from its acceptable list.

Randomly assign students to successive days of jobs. These could be placed together, or one at a time. Check if the student  has done the jobs before If an auxiliary job's list is empty, refill it with student names.

When using random, make sure to follow the [numpy best practice](https://numpy.org/doc/stable/reference/random/index.html) by creating an instance of a random number generator, rather than the legacy `seed()`.

- make an empty dataframe
- for each day, fill in all the speaker and leadership roles
    - check for duplicates in day
    - check for duplicates in multiple speaker roles
    - check for duplicates in multiple leadership roles, sans general evaluator
    - on the last day, fill in only as many roles as needed.
- starting from the beginning again, fill in all the auxiliary roles
    - checking for duplicates in day
    - checking for duplicates in role (less important, it's not preferable but acceptable to have a person repeat an auxiliary role

In [6]:
#fills in random roles for columns like prepared speaker 1,2,3.
def fill_repeated_roles(calendar, list_of_calendar_columns, role_dictionary, role_to_fill, day_dictionary, day):
    for role in list_of_calendar_columns:
                allowed_students = np.intersect1d(day_dictionary[day], role_dictionary[role_to_fill])
                #if the number of allowed students is less than 3, we've reached the end and only need to fill a few.
                #if that length is too short...
                if len(allowed_students) < 1:
                    #bail
                    return None
                else:
                    #pick out a student
                    this_student = rng.choice(allowed_students)
                    #remove the student from the day bucket
                    day_dictionary[day].remove(this_student)
                    #remove the student from the role bucket
                    role_dictionary[role_to_fill].remove(this_student)
                    #assign them that role
                    calendar.loc[day, role] = this_student
                    
#almost the same as above, but each role has its own entry in the role dictionary.
#Only changed roll_to_fill for role dictionary to role.
#this is used for the leadership roles
def fill_distinct_roles(calendar, list_of_calendar_columns, role_dictionary, day_dictionary, day):
    for role in list_of_calendar_columns:
                allowed_students = np.intersect1d(day_dictionary[day], role_dictionary[role])
                #if the number of allowed students is less than 3, we've reached the end and only need to fill a few.
                #if that length is too short...
                if len(allowed_students) < 1:
                    #bail
                    return None
                else:
                    #pick out a student
                    this_student = rng.choice(allowed_students)
                    #remove the student from the day bucket
                    day_dictionary[day].remove(this_student)
                    #remove the student from the role bucket
                    role_dictionary[role].remove(this_student)
                    #assign them that role
                    calendar.loc[day, role] = this_student

In [7]:
#this cell copied from above, with a larger num_days

num_days = 7
num_students = 23

students = 'abcdefghijklmnopqrstuvwxyz'[:num_students]

full_student_list = [
        char for char in students
]

speaker_roles = ['Prepared Speaker',
                'Impromptu Speaker',
                'Evaluator'
]

prepared_speakers = ['Prepared Speaker 1',
                     'Prepared Speaker 2',
                     'Prepared Speaker 3'
]

evaluators = [
    'Evaluator 1',
    'Evaluator 2',
    'Evaluator 3'
]

impromptu_speakers = [
    'Impromptu Speaker 1',
    'Impromptu Speaker 2',
    'Impromptu Speaker 3'    
]

speakers = ['Prepared Speaker 1',
           'Impromptu Speaker 1',
           'Evaluator 1',
            'Prepared Speaker 2',
            'Impromptu Speaker 2',
            'Evaluator 2',
            'Prepared Speaker 3',
            'Impromptu Speaker 3',
            'Evaluator 3'
]

#removed general evaluator!
leaders = [
            'Toastmaster',
            'President',
            'Table Topics Master'
]

general_evaluators = [
    'General Evaluator'
]

auxiliary = ['Greeter',
            'Joke Master',
            'Timer',
            'Grammarian',
            'Word of the Day',
            'Ah Counter',
            'Ballot Counter',
            'Thought of the day',
            'Sergeant at arms',
            'Stand-in'
]

In [19]:
# make an rng
seed = 20221
rng = np.random.default_rng(seed)

# search settings.
# How many times to try filling a day
day_fill_counter_max = 1000
# whether to print progress
verbose = False

In [20]:
cal = pd.DataFrame(columns = speakers + leaders + general_evaluators + auxiliary, index = range(num_days))
cal.index.rename('day', inplace=True)


student_buckets_by_day = {
    day:full_student_list.copy()
    for day in cal.index
}

#allowed students by role
student_buckets_by_role = {
    role:full_student_list.copy()
    for role in speaker_roles + ['Leaders'] + general_evaluators + auxiliary
}

In [21]:

# - make an empty dataframe
# - for each day, fill in all the speaker and leadership roles
#     - check for duplicates in day
#     - check for duplicates in multiple speaker roles
#     - check for duplicates in a set of opponents
#     - check for duplicates in multiple leadership roles, sans general evaluator
#     - on the last day, fill in only as many roles as needed.
# - starting from the beginning again, fill in all the auxiliary roles
#     - checking for duplicates in day
#     - checking for duplicates in role (less important, it's not preferable but acceptable to have a person repeat an auxiliary role


######
# fill roles
######
for day in cal.index:
    print(f'Filling speaker and leadership roles for day {day}.')
    #keep track whether we're done with the day at hand.
    day_filled = False
    #keep track of how many times we've attempted to fill the day. This is a random process and it's possible it won't work
    day_fill_counter = 0
    # keep track of original role buckets so they can be reset if it is misfilled
    this_day_original_role_buckets = student_buckets_by_role
    # until we're done 
    while not day_filled and day_fill_counter < day_fill_counter_max:
        day_fill_counter += 1
        #refill day's availability bucket. All students are available to be assigned roles from the day's POV.
        student_buckets_by_day[day] = list(full_student_list)
        #refill role buckets
        student_buckets_by_role = this_day_original_role_buckets
        #fill the prepared speaker roles
        fill_repeated_roles(cal, prepared_speakers, student_buckets_by_role, 'Prepared Speaker', student_buckets_by_day, day)
        #fill the impromptu speaker roles
        fill_repeated_roles(cal, impromptu_speakers, student_buckets_by_role, 'Impromptu Speaker', student_buckets_by_day, day)
        #fill the evaluator roles
        fill_repeated_roles(cal, evaluators, student_buckets_by_role, 'Evaluator', student_buckets_by_day, day)
        #fill the leadership roles
        fill_repeated_roles(cal, leaders, student_buckets_by_role, 'Leaders', student_buckets_by_day, day)
        #fill the general evaluators
        fill_distinct_roles(cal, general_evaluators, student_buckets_by_role, student_buckets_by_day, day)
        #fill the auxiliary roles
        fill_distinct_roles(cal, auxiliary, student_buckets_by_role, student_buckets_by_day, day)
        #check if there are no repeats in the day for the filled roles so far
        no_day_repeats = check_day_repeats(cal.loc[:day,speakers])
        #same for jobs
        no_job_repeats = check_job_repeats(cal.loc[:day,speakers])
        if no_day_repeats and no_job_repeats:
            day_filled = True
            print(f'found a solution for day {day}.')
    if not day_filled:
        print(f'Failed to fill day {day} in {day_fill_counter_max} iterations.')

        
        
        
        
###################
#attempt to fill in remaining speaker roles.
###################
cal[['Prepared Speaker 4','Impromptu Speaker 4','Evaluator 4']] = np.nan
# are there any stand-ins + auxiliary roles that are any of these roles?

# replace any stand-ins left to impromptu-speak as that day's 4th impromptu speaker
for leftover_student in student_buckets_by_role['Impromptu Speaker']:
    for day in cal.index:
        if cal.loc[day, 'Stand-in'] == leftover_student:
            cal.loc[day, 'Impromptu Speaker 4'] = leftover_student
            student_buckets_by_role['Impromptu Speaker'].remove(leftover_student)
# replace any stand-ins left to prepared_speak as that day's 4th prepared speaker.
# look for days where the evaluator candidate is doing a support role.
for leftover_student in student_buckets_by_role['Prepared Speaker']:
    for day in cal.index:
        #check if any of the leftover evaluators are support roles
        support_eval_intersection = np.intersect1d(student_buckets_by_role['Evaluator'], cal.loc[day, auxiliary])
        if cal.loc[day, 'Stand-in'] == leftover_student and len(support_eval_intersection) > 0:
            cal.loc[day, 'Prepared Speaker 4'] = leftover_student
            student_buckets_by_role['Prepared Speaker'].remove(leftover_student)
            cal.loc[day, 'Evaluator 4'] = support_eval_intersection[0]
            student_buckets_by_role['Evaluator'].remove(support_eval_intersection[0])
    #if that didn't work, just look for days where both are in support roles.
for leftover_student in student_buckets_by_role['Prepared Speaker']:
    for day in cal.index:
        support_eval_intersection = np.intersect1d(student_buckets_by_role['Evaluator'], cal.loc[day, auxiliary])
        if (leftover_student in cal.loc[day, auxiliary].values) and (len(support_eval_intersection) > 0):
            #print(f'filling {leftover_student}, {support_eval_intersection[0]}')
            cal.loc[day, 'Prepared Speaker 4'] = leftover_student
            student_buckets_by_role['Prepared Speaker'].remove(leftover_student)
            cal.loc[day, 'Evaluator 4'] = support_eval_intersection[0]
            student_buckets_by_role['Evaluator'].remove(support_eval_intersection[0])
# attempt to fill any remaining impromptu speakers on days they are doing auxiliary roles.
for leftover_student in student_buckets_by_role['Impromptu Speaker'].copy():
    student_placed = False
    print(f'looking for {leftover_student}')
    for day in cal.index:
        if (leftover_student in cal.loc[day, auxiliary].values) and cal.isna().loc[day,'Prepared Speaker 4'] and cal.isna().loc[day,'Impromptu Speaker 4'] and not student_placed:
            print(f'found {leftover_student} for impromptu speaker 4 on day {day}.')
            cal.loc[day, 'Impromptu Speaker 4'] = leftover_student
            student_buckets_by_role['Impromptu Speaker'].remove(leftover_student)
            student_placed = True
            
#run some checks
if cal[speakers + leaders + general_evaluators + auxiliary].isna().sum().sum() > 0:
    print(f'There are some non-4th speaker assignments without a student.')
for role in student_buckets_by_role:
    if role in speaker_roles and len(student_buckets_by_role[role]) > 0:
        print(f'There are unassigned {role} roles.')
if not check_speaker_evaluator_repeats(cal):
    print(f'There are some repeat prepared speaker / evaluator pairs.')
if len(student_buckets_by_role['Prepared Speaker']) > 0:
    print(f'There are students who have not been Prepared Speaker.')
if len(student_buckets_by_role['Impromptu Speaker']) > 0:
    print(f'There are students who have not been Impromptu Speaker.')
if len(student_buckets_by_role['Evaluator']) > 0:
    print(f'There are students who have not been Evaluator.')

Filling speaker and leadership roles for day 0.
found a solution for day 0.
Filling speaker and leadership roles for day 1.
found a solution for day 1.
Filling speaker and leadership roles for day 2.
found a solution for day 2.
Filling speaker and leadership roles for day 3.
found a solution for day 3.
Filling speaker and leadership roles for day 4.
found a solution for day 4.
Filling speaker and leadership roles for day 5.
found a solution for day 5.
Filling speaker and leadership roles for day 6.
found a solution for day 6.
looking for a
found a for impromptu speaker 4 on day 3.
looking for w
found w for impromptu speaker 4 on day 4.


In [22]:
23//3

7

In [23]:
cal

,Prepared Speaker 1,Impromptu Speaker 1,Evaluator 1,Prepared Speaker 2,Impromptu Speaker 2,Evaluator 2,Prepared Speaker 3,Impromptu Speaker 3,Evaluator 3,Toastmaster,President,Table Topics Master,General Evaluator,Greeter,Joke Master,Timer,Grammarian,Word of the Day,Ah Counter,Ballot Counter,Thought of the day,Sergeant at arms,Stand-in,Prepared Speaker 4,Impromptu Speaker 4,Evaluator 4
day,,,,,,,,,,,,,,,,,,,,,,,,,,
0,i,n,c,q,p,r,e,f,a,w,o,s,v,m,g,u,j,d,l,h,b,t,k,NaN,NaN,NaN
1,m,j,d,c,o,v,p,s,e,i,u,n,t,w,l,a,b,k,h,q,r,f,g,g,NaN,w
2,d,g,s,j,e,h,u,r,q,f,t,a,l,c,o,m,v,b,i,n,k,w,p,o,NaN,i
3,f,h,o,b,i,p,l,u,j,q,r,g,s,e,m,k,w,a,c,t,v,n,d,NaN,a,NaN
4,h,k,l,v,b,g,n,m,t,d,p,j,e,u,i,c,a,s,o,w,q,r,f,NaN,w,NaN
5,t,c,k,s,v,f,r,l,m,b,e,h,n,a,j,p,d,w,g,o,u,i,q,NaN,NaN,NaN
6,k,t,b,a,d,u,w,q,n,l,v,c,h,r,e,s,o,p,f,i,g,j,m,NaN,NaN,NaN


In [13]:
name_list = '''HIROKI
MISAKI
REINA
MOMOKO
MOEKA
RYOKO
STEVE
RENA
MAO
SHIORI
SAE F.
SAE N.
KEINA
PRECIOUS
MANAMI
SARINA
MIWAKO
MANAMI
SAYAKA
LONG
REBECCA
RIO
HANNAH'''

In [111]:
name_list = name_list.split('\n')

In [24]:
class_names = {
    full_student_list[index]:name_list[index]
    for index in range(len(full_student_list))
}
class_names

{'a': 'HIROKI',
 'b': 'MISAKI',
 'c': 'REINA',
 'd': 'MOMOKO',
 'e': 'MOEKA',
 'f': 'RYOKO',
 'g': 'STEVE',
 'h': 'RENA',
 'i': 'MAO',
 'j': 'SHIORI',
 'k': 'SAE F.',
 'l': 'SAE N.',
 'm': 'KEINA',
 'n': 'PRECIOUS',
 'o': 'MANAMI',
 'p': 'SARINA',
 'q': 'MIWAKO',
 'r': 'MANAMI',
 's': 'SAYAKA',
 't': 'LONG',
 'u': 'REBECCA',
 'v': 'RIO',
 'w': 'HANNAH'}

In [101]:
final_schedule = cal.replace(class_names)
final_schedule

,Prepared Speaker 1,Impromptu Speaker 1,Evaluator 1,Prepared Speaker 2,Impromptu Speaker 2,Evaluator 2,Prepared Speaker 3,Impromptu Speaker 3,Evaluator 3,Toastmaster,President,Table Topics Master,General Evaluator,Greeter,Joke Master,Timer,Grammarian,Word of the Day,Ah Counter,Ballot Counter,Thought of the day,Sergeant at arms,Stand-in,Prepared Speaker 4,Impromptu Speaker 4,Evaluator 4
day,,,,,,,,,,,,,,,,,,,,,,,,,,
0,MAO,PRECIOUS,REINA,MIWAKO,SARINA,MANAMI,MOEKA,RYOKO,HIROKI,HANNAH,MANAMI,SAYAKA,RIO,KEINA,STEVE,REBECCA,SHIORI,MOMOKO,SAE N.,RENA,MISAKI,LONG,SAE F.,NaN,NaN,NaN
1,KEINA,SHIORI,MOMOKO,REINA,MANAMI,RIO,SARINA,SAYAKA,MOEKA,MAO,REBECCA,PRECIOUS,LONG,HANNAH,SAE N.,HIROKI,MISAKI,SAE F.,RENA,MIWAKO,MANAMI,RYOKO,STEVE,STEVE,NaN,HANNAH
2,MOMOKO,STEVE,SAYAKA,SHIORI,MOEKA,RENA,REBECCA,MANAMI,MIWAKO,RYOKO,LONG,HIROKI,SAE N.,REINA,MANAMI,KEINA,RIO,MISAKI,MAO,PRECIOUS,SAE F.,HANNAH,SARINA,MANAMI,NaN,MAO
3,RYOKO,RENA,MANAMI,MISAKI,MAO,SARINA,SAE N.,REBECCA,SHIORI,MIWAKO,MANAMI,STEVE,SAYAKA,MOEKA,KEINA,SAE F.,HANNAH,HIROKI,REINA,LONG,RIO,PRECIOUS,MOMOKO,NaN,HIROKI,NaN
4,RENA,SAE F.,SAE N.,RIO,MISAKI,STEVE,PRECIOUS,KEINA,LONG,MOMOKO,SARINA,SHIORI,MOEKA,REBECCA,MAO,REINA,HIROKI,SAYAKA,MANAMI,HANNAH,MIWAKO,MANAMI,RYOKO,NaN,HANNAH,NaN
5,LONG,REINA,SAE F.,SAYAKA,RIO,RYOKO,MANAMI,SAE N.,KEINA,MISAKI,MOEKA,RENA,PRECIOUS,HIROKI,SHIORI,SARINA,MOMOKO,HANNAH,STEVE,MANAMI,REBECCA,MAO,MIWAKO,NaN,NaN,NaN
6,SAE F.,LONG,MISAKI,HIROKI,MOMOKO,REBECCA,HANNAH,MIWAKO,PRECIOUS,SAE N.,RIO,REINA,RENA,MANAMI,MOEKA,SAYAKA,MANAMI,SARINA,RYOKO,MAO,STEVE,SHIORI,KEINA,NaN,NaN,NaN


In [102]:
final_schedule

,Prepared Speaker 1,Impromptu Speaker 1,Evaluator 1,Prepared Speaker 2,Impromptu Speaker 2,Evaluator 2,Prepared Speaker 3,Impromptu Speaker 3,Evaluator 3,Toastmaster,President,Table Topics Master,General Evaluator,Greeter,Joke Master,Timer,Grammarian,Word of the Day,Ah Counter,Ballot Counter,Thought of the day,Sergeant at arms,Stand-in,Prepared Speaker 4,Impromptu Speaker 4,Evaluator 4
day,,,,,,,,,,,,,,,,,,,,,,,,,,
0,MAO,PRECIOUS,REINA,MIWAKO,SARINA,MANAMI,MOEKA,RYOKO,HIROKI,HANNAH,MANAMI,SAYAKA,RIO,KEINA,STEVE,REBECCA,SHIORI,MOMOKO,SAE N.,RENA,MISAKI,LONG,SAE F.,NaN,NaN,NaN
1,KEINA,SHIORI,MOMOKO,REINA,MANAMI,RIO,SARINA,SAYAKA,MOEKA,MAO,REBECCA,PRECIOUS,LONG,HANNAH,SAE N.,HIROKI,MISAKI,SAE F.,RENA,MIWAKO,MANAMI,RYOKO,STEVE,STEVE,NaN,HANNAH
2,MOMOKO,STEVE,SAYAKA,SHIORI,MOEKA,RENA,REBECCA,MANAMI,MIWAKO,RYOKO,LONG,HIROKI,SAE N.,REINA,MANAMI,KEINA,RIO,MISAKI,MAO,PRECIOUS,SAE F.,HANNAH,SARINA,MANAMI,NaN,MAO
3,RYOKO,RENA,MANAMI,MISAKI,MAO,SARINA,SAE N.,REBECCA,SHIORI,MIWAKO,MANAMI,STEVE,SAYAKA,MOEKA,KEINA,SAE F.,HANNAH,HIROKI,REINA,LONG,RIO,PRECIOUS,MOMOKO,NaN,HIROKI,NaN
4,RENA,SAE F.,SAE N.,RIO,MISAKI,STEVE,PRECIOUS,KEINA,LONG,MOMOKO,SARINA,SHIORI,MOEKA,REBECCA,MAO,REINA,HIROKI,SAYAKA,MANAMI,HANNAH,MIWAKO,MANAMI,RYOKO,NaN,HANNAH,NaN
5,LONG,REINA,SAE F.,SAYAKA,RIO,RYOKO,MANAMI,SAE N.,KEINA,MISAKI,MOEKA,RENA,PRECIOUS,HIROKI,SHIORI,SARINA,MOMOKO,HANNAH,STEVE,MANAMI,REBECCA,MAO,MIWAKO,NaN,NaN,NaN
6,SAE F.,LONG,MISAKI,HIROKI,MOMOKO,REBECCA,HANNAH,MIWAKO,PRECIOUS,SAE N.,RIO,REINA,RENA,MANAMI,MOEKA,SAYAKA,MANAMI,SARINA,RYOKO,MAO,STEVE,SHIORI,KEINA,NaN,NaN,NaN


In [103]:
final_schedule.to_csv('toastmasters.csv')